In [1]:
!pip install -U transformers datasets==3.2.0 openai lancedb lance FlagEmbedding "tantivy>=0.20.1" -qq

# NOTE: If there is an import error, restart and run the notebook again

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.8/34.8 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.5 MB/s eta 0:00:00


In [2]:
import os
import lancedb
import re
import pandas as pd
import random

from datasets import load_dataset

import torch
import gc

import lance


import os

import lancedb
import openai
from lancedb.embeddings import get_registry
from lancedb.pydantic import LanceModel, Vector


os.environ["OPENAI_API_KEY"] = "sk-proj-...."


embeddings = get_registry().get("openai").create()

# Load `Chunks` of data from [BeIR Dataset](https://huggingface.co/datasets/BeIR/scidocs)

Note: This is a dataset built specially for retrieval tasks to see how good your search is working

In [ ]:
import os
from datasets import config


queries = load_dataset("BeIR/scidocs", "queries")["queries"].to_pandas()
full_docs = (
    load_dataset("BeIR/scidocs", "corpus")["corpus"].to_pandas().dropna(subset="text")
)

docs = full_docs.head(64)  # just random samples for faster embed demo
docs["num_words"] = docs["text"].apply(
    lambda x: len(x.split())
)  # Insert some Metadata for a more "HYBRID" search
docs.sample(5)

README.md: 0.00B [00:00, ?B/s]

scidocs.py: 0.00B [00:00, ?B/s]

0000.parquet:   0%|          | 0.00/93.3k [00:00<?, ?B/s]

Generating queries split:   0%|          | 0/1000 [00:00<?, ? examples/s]

0000.parquet:   0%|          | 0.00/19.0M [00:00<?, ?B/s]

Generating corpus split:   0%|          | 0/25657 [00:00<?, ? examples/s]

/tmp/ipython-input-4-751017049.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docs["num_words"] = docs["text"].apply(


,_id,title,text,num_words
15,506d4ca228f81715946ed1ad8d9205fad20fddfe,Measuring pictorial balance perception at firs...,"According to art theory, pictorial balance act...",212
34,105a0b3826710356e218685f87b20fe39c64c706,Opinion observer: analyzing and comparing opin...,The Web has become an excellent source for gat...,208
49,abd0478f1572d8ecdca4738df3e4b3bd116d7b42,Dispositional Factors in Internet Use: Persona...,This study directly tests the effect of person...,83
46,e645cbd3aaeab56858f1e752677b8792d7377d14,BCSAT : A Benchmark Corpus for Sentiment Analy...,The presented work aims at generating a system...,126
54,30f46fdfe1fdab60bdecaa27aaa94526dfd87ac1,Real-Time 3D Reconstruction and 6-DoF Tracking...,We propose a method which can perform real-tim...,150


# Build New Table

In [5]:
!rm -rf ./db

In [6]:
class Documents(LanceModel):
    vector: Vector(embeddings.ndims()) = embeddings.VectorField()
    text: str = embeddings.SourceField()
    title: str
    num_words: int


data = docs.apply(
    lambda row: {
        "title": row["title"],
        "text": row["text"],
        "num_words": row["num_words"],
    },
    axis=1,
).values.tolist()

db = lancedb.connect("./db")
table = db.create_table("documents", schema=Documents)

table.add(data)  # ingest docs with auto-vectorization
table.create_fts_index("text")  # Create a fts index before the hybrid search

In [7]:
table.search(
    "To confuse the AI and DNN embedding, let's put random terms from other sentences- automation training test memory?",
    query_type="fts",
).limit(5).to_pandas()

,vector,text,title,num_words,_score
0,"[-0.012233193, -0.0030392595, 0.022818677, -0....","Sociological and technical difficulties, such ...",Hipikat: a project memory for software develop...,210,48.126713
1,"[-0.00843631, -0.013696598, 0.0075827544, -0.0...","Iron, the most ubiquitous of the transition me...","Synthesis, properties, and applications of iro...",158,47.561138
2,"[0.005474513, -0.0017037347, 0.009329896, -0.0...",Mobility prediction is one of the most essenti...,A data mining approach for location prediction...,135,43.397488
3,"[-0.010289601, -0.0014841412, -0.0050512585, -...",It's not surprisingly when entering this site ...,Genetic Fuzzy Systems - Evolutionary Tuning an...,100,36.493389
4,"[-0.03584866, -0.00017609358, 0.00922457, -0.0...",We introduce a long short-term memory recurren...,Speech-driven 3 D Facial Animation with Implic...,171,27.971348


In [8]:
table.search(
    "To confuse the AI and DNN embedding, let's put random terms from other sentences- automation training test memory?",
    query_type="vector",
).limit(10).to_pandas()

,vector,text,title,num_words,_distance
0,"[-0.02901112, -0.0019353012, 0.010076279, -0.0...",Classifying short texts to one category or clu...,Using deep learning for short text understanding,187,0.354657
1,"[-0.01352904, -0.0019552337, 0.0095874, -0.030...",a r t i c l e i n f o a b s t r a c t We prese...,"BabelNet: The automatic construction, evaluati...",133,0.402470
2,"[-0.027341157, -0.0001916921, 0.017680977, -0....",We introduce a technique for augmenting neural...,Deep Voice 2 : Multi-Speaker Neural Text-to-Sp...,151,0.417746
3,"[-0.03584866, -0.00017609358, 0.00922457, -0.0...",We introduce a long short-term memory recurren...,Speech-driven 3 D Facial Animation with Implic...,171,0.421450
4,"[-0.010492763, -0.006538273, 0.01853196, -0.03...",The presented work aims at generating a system...,BCSAT : A Benchmark Corpus for Sentiment Analy...,126,0.429735
5,"[-0.03796007, 0.010790204, 0.029059827, -0.034...",This paper proposes quiz-style information pre...,Effects of quiz-style information presentation...,141,0.442949
6,"[-0.03084091, -0.002591715, 0.010260329, -0.01...",This report summarizes the objectives and eval...,SemEval-2015 Task 11: Sentiment Analysis of Fi...,112,0.444655
7,"[-0.00980282, 0.007281899, 0.009843921, -0.031...",This paper presents a simple unsupervised lear...,Thumbs Up or Thumbs Down? Semantic Orientation...,163,0.448142
8,"[-0.027524939, 0.017282786, 0.016536668, -0.01...","With the development of online advertisements,...",Clickbait Convolutional Neural Network,160,0.448332
9,"[-0.016280893, 0.010584505, 0.030072825, -0.01...",Theories on the functions of the hippocampal s...,"Memory, navigation and theta rhythm in the hip...",130,0.453260


## Perform inbuilt Hybrid Search
They have some off the shelf functionalities and a way to implement the custom Re-Ranking and Filtering Function here [Implement Custom Rerankers](https://lancedb.github.io/lancedb/hybrid_search/#building-custom-rerankers)

In [9]:
from lancedb.rerankers import LinearCombinationReranker

reranker = LinearCombinationReranker(
    weight=0.7
)  # Weight = 0 Means pure Text Search (BM-25) and 1 means pure Sementic (Vector) Search

table.search(
    "To confuse the AI and DNN embedding, let's put random terms from other sentences- automation training test memory?",
    query_type="hybrid",
).rerank(reranker=reranker).limit(5).to_pandas()

,vector,text,title,num_words,_relevance_score
0,"[-0.03584866, -0.00017609358, 0.00922457, -0.0...",We introduce a long short-term memory recurren...,Speech-driven 3 D Facial Animation with Implic...,171,0.921843
1,"[-0.010289601, -0.0014841412, -0.0050512585, -...",It's not surprisingly when entering this site ...,Genetic Fuzzy Systems - Evolutionary Tuning an...,100,0.873155
2,"[0.005474513, -0.0017037347, 0.009329896, -0.0...",Mobility prediction is one of the most essenti...,A data mining approach for location prediction...,135,0.770392
3,"[-0.00843631, -0.013696598, 0.0075827544, -0.0...","Iron, the most ubiquitous of the transition me...","Synthesis, properties, and applications of iro...",158,0.708418
4,"[-0.010492763, -0.006538273, 0.01853196, -0.03...",The presented work aims at generating a system...,BCSAT : A Benchmark Corpus for Sentiment Analy...,126,0.700000


## Build custom Filtering Function

By passing the `pandas.query` style, filtering, we will do the following 2 things:

1. Remove all the rows which contain a specific term, in out case, `"dual-band"`
2. Keep only the rows which have `num_words > 100`

In [10]:
from typing import List, Union
import pandas as pd
import pyarrow as pa


class MofidifiedLinearReranker(LinearCombinationReranker):
    def __init__(self, filters: Union[str, List[str]], **kwargs):
        super().__init__(**kwargs)
        filters = filters if isinstance(filters, list) else [filters]
        self.filters = filters

    def rerank_hybrid(
        self, query: str, vector_results: pa.Table, fts_results: pa.Table
    ) -> pa.Table:
        combined_result = super().rerank_hybrid(query, vector_results, fts_results)
        df = combined_result.to_pandas()
        for filter in self.filters:
            df = df.query(
                "(not text.str.contains(@filter)) & (num_words > 150) "
            )  # THIS is where you implement your filters. You can hard code or pass dynamically too

        return pa.Table.from_pandas(df)


modified_reranker = MofidifiedLinearReranker(filters=["dual-band"])

table.search(
    "To confuse the AI and DNN embedding, let's put random terms from other sentences- automation training test memory?",
    query_type="hybrid",
).rerank(reranker=modified_reranker).limit(7).to_pandas()

,vector,text,title,num_words,_relevance_score
0,"[-0.027824033, -0.00014784517, 0.012952094, -0...",In this paper a novel approach is proposed to ...,Text mining of news-headlines for FOREX market...,621,1.000000
1,"[0.008277873, 0.023134368, -0.0037977214, -0.0...",This paper introduces a new compositional fram...,Dissecting and Reassembling Color Correction A...,237,0.902053
4,"[-0.00843631, -0.013696598, 0.0075827544, -0.0...","Iron, the most ubiquitous of the transition me...","Synthesis, properties, and applications of iro...",158,0.705130
5,"[-0.03584866, -0.00017609358, 0.00922457, -0.0...",We introduce a long short-term memory recurren...,Speech-driven 3 D Facial Animation with Implic...,171,0.701210
7,"[-0.012233193, -0.0030392595, 0.022818677, -0....","Sociological and technical difficulties, such ...",Hipikat: a project memory for software develop...,210,0.700000
10,"[-0.027341157, -0.0001916921, 0.017680977, -0....",We introduce a technique for augmenting neural...,Deep Voice 2 : Multi-Speaker Neural Text-to-Sp...,151,0.489522
12,"[-0.02901112, -0.0019353012, 0.010076279, -0.0...",Classifying short texts to one category or clu...,Using deep learning for short text understanding,187,0.000000
